In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/korean-hate-speech-detection/test.news_title.txt
/kaggle/input/korean-hate-speech-detection/test.hate.no_label.csv
/kaggle/input/korean-hate-speech-detection/dev.hate.csv
/kaggle/input/korean-hate-speech-detection/dev.news_title.txt
/kaggle/input/korean-hate-speech-detection/unlabeled_comments.txt
/kaggle/input/korean-hate-speech-detection/unlabeled_comments.news_title.txt
/kaggle/input/korean-hate-speech-detection/train.hate.csv
/kaggle/input/korean-hate-speech-detection/train.news_title.txt


In [2]:
!pip install -q -U transformers 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.14.0 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.16.4 which is incompatible.
allennlp 2.7.0 requires transformers<4.10,>=4.1, but you have transformers 4.30.2 which is incompatible.


In [3]:
from transformers import pipeline

pipe = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-hate', device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
pipe("안녕하세요")

[{'label': 'none', 'score': 0.9638350009918213}]

In [5]:
df = pd.read_csv('/kaggle/input/korean-hate-speech-detection/test.hate.no_label.csv')

In [6]:
df

,comments
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ
1,둘다 넘 좋다~행복하세요
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요
4,장현승 얘도 참 이젠 짠하다...
...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...
972,입에 손가릭이 10개 있으니 징그럽다


In [7]:
df['label'] = df['comments'].map(lambda x: pipe(x)[0]['label'])

/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1084: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [8]:
df.label.value_counts()

none         405
offensive    334
hate         235
Name: label, dtype: int64

In [9]:
LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

In [37]:
df['label'] = df['label'].map(lambda x: LABEL_DIC[x])

SyntaxError: invalid syntax (864088669.py, line 1)

In [11]:
df.to_csv('./submission.csv', index=None)

In [12]:
!head -n 10 submission.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
comments,label
ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,1
둘다 넘 좋다~행복하세요,0
근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
장현승 얘도 참 이젠 짠하다...,0
신선하게 웃긴다ㅋㅋㅋ역시 동엽신~~!! 장소연님은 진짜 조선족인가 착각할정도로 말투가 리얼하네요,1
누군데 얘네?,1
"하자 인생들 모아다가 방송에 내보내고, 덜 하자가 교정해서 장사 풀리게 해주는 감동 스토리 백하자의 골목식당. 호텔 말고 그냥 하자 거리를 하나 열어서 거기다 하자 인생들 교화소를 만들지... 왜 저러고 살까...",2
진짜 라디오 스타 노래한거 보세요 홍진영은비비지도 못함,0


In [13]:
print('finish')

finish


In [66]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Hugging Face의 사전 훈련된 KoBERT 모델 및 토크나이저를 사용
model_name = "monologg/kobert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 문장을 입력으로 받아 토큰화하고 모델로 분류
def classify_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)[0].tolist()
    
    # offensive 여부 판단
    offensive_threshold = 0.5
    if probabilities[1] > offensive_threshold:
        return "offensive"
    elif probabilities[0] > offensive_threshold:
        return "hate"
    else:
        return "None"

# 사용 예시
sentence = "이 문장은 정말 나쁘다!"
result = classify_sentence(sentence)
print(f"입력 문장: {sentence}")
print(f"분류 결과: {result}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


입력 문장: 이 문장은 정말 나쁘다!
분류 결과: hate


In [67]:
df_test = pd.read_csv('/kaggle/input/korean-hate-speech-detection/test.hate.no_label.csv')

In [68]:
numOfData = len(df_test["comments"])
# numOfData = 974

LABEL_DIC = {
    'None': 0,
    'offensive': 1,
    'hate': 2,
}

for i in range(numOfData):
    sentence = df_test["comments"][i]
    ans = classify_sentence(sentence)
    label = LABEL_DIC.get(ans, 0)
    label = int(label)
    df_test.at[i, 'label'] = label

# 데이터프레임을 CSV 파일로 저장

In [70]:
print(label)
df_test['label'][0]

# float_format='%.0f' 로 해결 

1


2.0

In [71]:
df_test.to_csv('./submission_test_int.csv', index=None, float_format='%.0f')

# 성능: 0.25284

In [47]:
# # https://github.com/SKTBrain/KoBERT 

# first = df["comments"][0]

# classify_sentence(first)

# first_ans = df["label"][0]
# print(first_ans)


# # fail or success

# numOfData = len(df["comments"])
# # numOfData = 974

# success = 0
# fail = 0

# LABEL_DIC = {
#     'None': 0,
#     'offensive': 1,
#     'hate': 2,
# }

# for i in range (numOfData):
#     sentence = df["comments"][i]
#     ans = classify_sentence(sentence)
#     ans = [LABEL_DIC.get(x, 0) for x in ans]
    
#     if all(ans == df["label"][i]):
#         success = success + 1
#     else:
#         fail = fail + 1

# print(success)
# print(fail)

405
569


In [52]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer

# # Hugging Face의 사전 훈련된 T5 모델 및 토크나이저를 사용
# model_name = "t5-small"
# model = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer = T5Tokenizer.from_pretrained(model_name)

# # 문장을 입력으로 받아 토큰화하고 모델로 분류
# def classify_sentence(sentence):
#     # T5 모델은 입력 문장을 "classify: "라는 접두어로 붙여줘야 합니다.
#     input_text = "classify: " + sentence
#     inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
#     outputs = model.generate(**inputs)
#     decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#     # 분류 결과 확인
#     labels = ["hate", "offensive", "None"]
#     return decoded_output

# # 예제 문장을 분류해보기
# example_sentence = "이 문장은 모욕적이에요!"
# classification = classify_sentence(example_sentence)
# print(f"분류 결과: {classification}")

# # 결과가 이상하게 나옴
# # 분류 결과: Klassify:  !

분류 결과: Klassify:  !
